In [15]:
def cleanToken(t):
    t = t.replace("...", "")
    t = t.replace("tout.à.l'heure", "tout_à_l'heure")

    t=t.replace('<font color=green>','')
    t = t.replace('</font color=green>','')

    t = t.replace("<u>", "")
    t = t.replace("</u>", "")


    return t


In [16]:
import re
def alignTokLemmgloss(text, lemgloss, phonetic):
    '''
    takes three lines of an example, produces triples
    '''
    pattern = r'\[\[(\w+)\|\|(\w+)\]\]'
    lemgloss = re.sub(pattern, r'[[\1|\2]]', lemgloss)


    text_list = text.split("||")
    lemgloss_list = lemgloss.split("||")
    phon_list = phonetic.split("||")
    couple_list = []



    if any("KLT" in tok or "Sujet" in tok for tok in text_list) or any("KLT" in lem or "Sujet" in lem for lem in lemgloss_list):
        return couple_list

    text_list_stripped = [tok.strip() for tok in text_list]
    lemgloss_list_stripped = [lem.strip() for lem in lemgloss_list]
    phon_list_stripped = [phon.strip() for phon in phon_list]

    if "standard" in text.lower() or "standard" in lemgloss.lower() or "Équivalent" in text or "Graphie" in text:
        return couple_list

    if text_list_stripped and text_list_stripped[-1] == "":
        text_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "<elles>":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and lemgloss_list_stripped[-1] == "_":
        lemgloss_list_stripped.pop()
    if lemgloss_list_stripped and re.search(r'\d{4}', lemgloss_list_stripped[-1]):
        return couple_list
    if text_list_stripped and re.search(r'\d{4}', text_list_stripped[-1]):
        return couple_list
    if text_list_stripped and "colspan" in text_list_stripped[-1]:
        return couple_list
    if "[[*]]" in text_list_stripped:
        # this indicates that the sentence is ungrammatical, and we skip it
        return couple_list

    if len(text_list_stripped) == len(lemgloss_list_stripped):
        
        if phonetic=="None" :
            for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
                tok = cleanToken(tok)
                lemmgloss = cleanToken(lemmgloss)
                if tok and lemmgloss:
                    couple_list.append((tok, lemmgloss,'None'))
        elif phonetic !='None':
            for tok, lemmgloss,phone in zip(text_list_stripped, lemgloss_list_stripped,phon_list_stripped):
                tok = cleanToken(tok)
                lemmgloss = cleanToken(lemmgloss)
                phone=cleanToken(phone)
                if tok and lemmgloss and phone:
                    couple_list.append((tok, lemmgloss,phone))

    else:
        punctuation_marks = ['!', '.', ',', '?', ';', ':']
        for punctuation in punctuation_marks:
            if text_list_stripped and text_list_stripped[-1].endswith(punctuation) and not lemgloss_list_stripped[-1].endswith(punctuation):
                lemgloss_list_stripped.append(punctuation)

        if len(text_list_stripped) == len(lemgloss_list_stripped):
            if phonetic == "None":
                for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
                    tok = cleanToken(tok)
                    lemmgloss = cleanToken(lemmgloss)
                    if tok and lemmgloss:
                        couple_list.append((tok, lemmgloss, 'None'))
            elif phonetic != 'None':
                for tok, lemmgloss, phone in zip(text_list_stripped, lemgloss_list_stripped, phon_list_stripped):
                    tok = cleanToken(tok)
                    lemmgloss = cleanToken(lemmgloss)
                    phone = cleanToken(phone)
                    if tok and lemmgloss and phone:
                        couple_list.append((tok, lemmgloss, phone))
        else:
            for punctuation in punctuation_marks:
                if lemgloss_list_stripped and lemgloss_list_stripped[-1].endswith(punctuation) and not \
                text_list_stripped[-1].endswith(punctuation):
                    text_list_stripped.append(punctuation)

            if len(text_list_stripped) == len(lemgloss_list_stripped):
                if phonetic == "None":
                    for tok, lemmgloss in zip(text_list_stripped, lemgloss_list_stripped):
                        tok = cleanToken(tok)
                        lemmgloss = cleanToken(lemmgloss)
                        if tok and lemmgloss:
                            couple_list.append((tok, lemmgloss, 'None'))
                elif phonetic != 'None':
                    for tok, lemmgloss, phone in zip(text_list_stripped, lemgloss_list_stripped, phon_list_stripped):
                        tok = cleanToken(tok)
                        lemmgloss = cleanToken(lemmgloss)
                        phone = cleanToken(phone)
                        if tok and lemmgloss and phone:
                            couple_list.append((tok, lemmgloss, phone))

            else:
                with open("not_couple.txt", "a") as file:
                    file.write("text\n")
                    file.write(f"{text}\n")
                    file.write("lemgloss\n")
                    file.write(f"{lemgloss}\n")
    return couple_list

In [9]:
import re
def couple2tokobjects(triple):
    text, lem,phon = triple

    lem = re.sub(r"\[\[cardinal\|deux\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|deux\]\].F<sup>\[\[1\]\]</sup>", "[[div|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|quatre\]\]<sup>\[\[2\]\]</sup>", "[[peder|quatre]]", lem)

    lem = re.sub(r"<sup>\[\[2\]\]</sup> \[\[cardinal\|deux\]\]", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|deux\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|2\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinal\|cent\]\]", "[[gant|cent]]", lem)
    lem = re.sub(r"\[\[cardinal\|cent\]\]<sup>\[\[1\]\]</sup>", "[[kant|cent]]", lem)
    lem = re.sub(r"<sup>\[\[2\]\]</sup>\[\[cardinal\|cent\]\]", "[[c'hant|cent]]", lem)
    lem = re.sub(r"\[\[cardinal\|mille\]\]<sup>\[\[1\]\]</sup>", "[[mil|mille]]", lem)
    lem = re.sub(r"\[\[cardinal\|50\]\]<sup>\[\[1\]\]</sup>", "[[50|50]]", lem)
    lem = re.sub(r"\[\[cardinaux\|deux\]\]<sup>\[\[1\]\]</sup>", "[[zaou|deux]]", lem)
    lem = re.sub(r"\[\[dek\|dix\]\]<sup>\[\[1\]\]</sup>", "[[dek|dix]]", lem)
    lem = re.sub(r"\[\[cardinaux\|dix\]\]", "[[dek|dix]]", lem)
    lem = re.sub(r"\[\[cardinaux\|2\]\]<sup>\[\[1\]\]</sup>", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinaux\|deux\]\]", "[[daou|deux]]", lem)
    lem = re.sub(r"\[\[cardinaux\|mille\]\]<sup>\[\[1\]\]</sup>", "[[mil|mille]]", lem)

    lem = re.sub(r"\[\[cardinal\|50\]\]", "[[50|50]]", lem)

    if 'diou' in text:
        lem = re.sub(r"\[\[cardinal\|deux\.F\]\]<sup>\[\[1\]\]</sup>", "[[diou|deux.F]]", lem)
    if 'vil' in text:
        lem = re.sub(r"<sup>\[\[1\]\]</sup>\[\[cardinal\|mille\]\]", "[[vil|mille]]", lem)
    if 'diouig' in text:
        lem = re.sub(r"\[\[cardinal\|2\]\]", "[[diou|2]]", lem)
    if "daou-ha-daou" in text:
        lem = re.sub(r"\[\[cardinal\|deux\]\]", "[[daou|deux]]", lem)

    lem = re.sub(r'<sup>(\[\[[^\[\]]+\]\])(.*?)</sup>', lambda m: ''.join(f'<sup>{x}</sup>' for x in re.findall(r'\[\[[^\[\]]+\]\]', m.group(0))), lem)
    lem = re.sub('particule o','o',lem)

    lem = re.sub(r'<font color=green>.*?</font color=green>', '', lem)

    phon=re.sub('\[','',phon)
    phon = re.sub(']', '', phon)

    if '[[R]]' not in lem :
        match = re.search(r'<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', lem)
        if match:
            k = match.group(2).strip(']')
            lem = re.sub(r'<sup>\[\[(?!(1|4)\]\]).*?</sup>', '[[R|_]]', lem)

        if "particule o" not in lem:
            lem = re.sub(r'<sup>\[\[4\]\]</sup>', '[[e|_]]', lem)

        lem = re.sub(r'<sup>\[\[1\]\]</sup>', '[[a|_]]', lem)


    if '[[R]]' in lem and '<sup>' in lem:
        match1 = re.search(r'\[\[R\]\]<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', lem)
        if match1:
            k = match1.group(2).strip(']')
            lem = re.sub(r'\[\[R\]\]<sup>\[\[(?!(1|4)\]\])(.*?)</sup>', '[[R|_]]', lem)

        lem = re.sub(r'\[\[R\]\]<sup>\[\[4\]\]</sup>', '[[e|_]]', lem)
        lem = re.sub(r'\[\[R\]\]<sup>\[\[1\]\]</sup>', '[[a|_]]', lem)

    if '[[R]]' in lem and '<sup>' not in lem:
        lem = re.sub(r'\[\[R\]\]','[[R|_]]',lem)
        k='R'

    if "'''" in text and '-'in lem and re.search(r'""".*?"""', text):
        extra_lem=re.search(r"'''(.*?)'''", text).group(1)
        match = re.search(r'(?<=\|{2}).*?-(.*?)(?=\|\|)', lem)

        extra_gloss = match.group(1).strip()
        lem = re.sub(r'-(.*?)\|\|', f'[[{extra_lem}|{extra_gloss}]] ||', lem)


    text_list = text.split('||')
    lem_list = lem.split('||')
    phone_list = phon.split('||')

    tok_objects = []

    for i in range(len(text_list)):
        tok = text_list[i].strip()
        lem_parts = lem_list[i].strip().split(']]')
        lem_sp = lem_list[i].strip().split('|')
        phonetic = phone_list[i].strip()

        import string

        tok_list = [part for part in tok.split(' ') if not all(char in string.punctuation for char in part)]
        phon_list = [part2 for part2 in phon.split(' ') if not all(char2 in string.punctuation for char2 in part2)]


        if len(lem_sp)== 1 and tok.startswith("'''") :
            tok_objects.append({
                'tok': tok.replace("'''",''),
                'lemma': tok.replace("'''",''),
                'gloss': lem_parts[0],
                'phonetic':phonetic.replace("'''",'')
            })


        elif len(lem_sp)==1 and "<sub>" in lem_sp[0] and ('SC' or 'VP' or 'PredP' or 'CP' or 'DP') in lem_sp[0]:
            match = re.search(r'<sub>\[\[(.*?)\]\]', lem_sp[0])
            if match:
                content = match.group(1)
                tok_objects.append({
                    'PhraseStructure': content
                })

        elif len(lem_sp)==1 and "<sub>" in lem_sp[0] and '[[' not in lem_sp[0]:
            match = re.search(r'<sub>(.*?)</sub>', lem_sp[0])
            if match:
                content = match.group(1)
                tok_objects.append({
                    'index': content
                })

        elif len(lem_sp) == 2 and '[[' in lem_sp[0]:

            gloss = lem_sp[1].rstrip('].')


            if 'cardinal' in lem_sp[0] and 'cardinal' not in tok:

                lem_sp[0]=tok
                tok_objects.append({
                    'tok': tok,
                    'lemma': tok,
                    'gloss': gloss,
                    'phonetic':phonetic if phonetic else '_'
                })
            if "cardinaux" in lem_sp[0] :
                lem_sp[0] = tok
                tok_objects.append({
                    'tok': tok,
                    'lemma': tok,
                    'gloss': gloss,
                    'phonetic':phonetic if phonetic else '_'
                })

            elif 'pronom in' in lem:
                lem_sp[0] = re.sub("\[\[pronom incorporé", "me",lem_sp[0]) if "moi" in gloss else ''
                lem_sp[0] = re.sub("\[\[pronom incorporé", "te", lem_sp[0]) if "toi" in gloss else ''
                lem_sp[0] = re.sub("\[\[pronom incorporé", "eñ", lem_sp[0]) if "lui" in gloss else ''
                lem_sp[0] = re.sub("\[\[pronom incorporé", "hi", lem_sp[0]) if "elle" in gloss else ''
                lem_sp[0] = re.sub("\[\[pronom incorporé", "hon", lem_sp[0]) if "nous" in gloss else ''
                lem_sp[0] = re.sub("\[\[pronom incorporé", "hoc'h-unan", lem_sp[0]) if "vous" in gloss else ''
                lem_sp[0] = re.sub("\[\[pronom incorporé", "int", lem_sp[0]) if "eux" in gloss else ''


                tok_objects.append({
                    'tok': tok,
                    'lemma': lem_sp[0],
                    'gloss': gloss,
                    'phonetic':phonetic if phonetic else '_'
                })
            else:
                gloss = lem_sp[1].rstrip('].')
                tok_objects.append({
                    'tok': tok,
                    'lemma': lem_sp[0].strip('[]'),
                    'gloss': gloss,
                    'phonetic':phonetic if phonetic else '_'
                })


        elif len(tok_list)==len(lem_parts)-1:
            for i in range(len(tok_list)):
                lem_inside=lem_parts[i].split('|')[0]
                gloss_inside=lem_parts[i].split('|')[-1]

                if 'numéraux' in lem_inside:
                    lem_inside="[["+tok

                if "cardinaux" in lem_inside:
                    lem_inside="[["+tok


                if '-ig' in lem_inside:
                    gloss_inside='petit'

                if 'cardinal' in lem_inside and 'cardinal' not in tok:
                    lem_inside=tok
                    pattern = r"[^\w\s]+"
                    lem_inside=re.sub(pattern,"",lem_inside)
                    lem_inside='[['+lem_inside

                if 'pronom incorporé' in lem_inside:
                    lem_inside=tok

                if len(tok_list) == len(phon_list):
                    tok_objects.append({'tok':tok_list[i],
                                        'lemma': lem_inside[2:],
                                        'gloss': gloss_inside,
                                        'phonetic':phon_list[i] if phon_list[i] else 'None'

                    })
                else:
                    tok_objects.append({'tok': tok_list[i],
                                        'lemma': lem_inside[2:],
                                        'gloss': gloss_inside

                                        })
                    if phonetic != 'None':
                        with open('error_phon.txt','a')as fi:
                            fi.write(f'text\n{text}\n')
                            fi.write(f'phonetic\n{phon}\n')
                            fi.write((f'______________\n'))




        else:
            tok_objects.append({
                'tok': tok,
                'lemma': '_',
                'gloss': '_',
                'phonetic':phonetic if phonetic else '_',
                'multiple': len(lem_parts) - 1})
            tok_obj = {'tok': tok, 'lemma': '_', 'gloss': '_','multiple':len(lem_parts)-1}
            tok_n = 1

            for j in range(len(lem_parts)):
                if '[[' in lem_parts[j]:
                    lemma_gloss = lem_parts[j].strip('[[').split('|')
                    lemma = lemma_gloss[0].split('[[')[-1].strip()

                    if re.search(r'^-\w+', lemma):
                        if lemma !='-ig':
                            tok_objects.append({
                                'tok': lemma.strip(),
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 and tok != '-ig' else ''
                            })

                        else:
                            tok_objects.append({
                                'tok': lemma.strip(),
                                'lemma': lemma.strip(),
                                'gloss': 'petit'
                            })




                    elif lemma_gloss == ['R', '_']:
                        if k !='+C':
                            tok_objects.append({
                                'tok': k,
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                            })

                        else:
                            tok_objects.append({
                                'tok': k,
                                'lemma': lemma.strip(),
                                'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else '',
                                                      'Epenthesis':'Yes'
                            })

                    elif lemma_gloss == ['a', '_']:
                        tok_objects.append({
                            'tok': '1',
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                        })

                    elif lemma_gloss == ['e', '_']:
                        tok_objects.append({
                            'tok': '4',
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''
                        })

                    elif 'cardinal' in lemma_gloss[0]:
                        tok_objects.append({
                            'tok': lemma_gloss[1],
                            'lemma': lemma_gloss[1],
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardinaux" in lemma_gloss[0] and 'cent' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'kant',
                            'lemma': 'kant',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif 'cardi' in lemma_gloss[0] and '9' in lemma_gloss[1]:

                        tok_objects.append({
                            'tok': 'nav',
                            'lemma': 'nav',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'trois' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'tri',
                            'lemma': 'tri',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'vingt' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'ugent',
                            'lemma': 'ugent',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and '2' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daou',
                            'lemma': 'daou',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )

                    elif "cardi" in lemma_gloss[0] and 'douze' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daouzek',
                            'lemma': 'daouzek',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif "cardi" in lemma_gloss[0] and 'deux' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'daou',
                            'lemma': 'daou',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )
                    elif "cardi" in lemma_gloss[0] and '3' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'tri',
                            'lemma': 'tri',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "cardi" in lemma_gloss[0] and 'un' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'unan',
                            'lemma': 'unan',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})

                    elif "pronom inco" in lemma_gloss[0] and 'lui' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'eñ',
                            'lemma': 'eñ',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'moi' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'me',
                            'lemma': 'me',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'toi' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'te',
                            'lemma': 'te',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'nous' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'hon',
                            'lemma': 'hon',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'elle' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': 'hi',
                            'lemma': 'hi',
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'vous' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "hoc'h-unan",
                            'lemma': "hoc'h-unan",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'eux' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "int",
                            'lemma': "int",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'il' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "enni",
                            'lemma': "enni",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})
                    elif "pronom inco" in lemma_gloss[0] and 'ça' in lemma_gloss[1]:
                        tok_objects.append({
                            'tok': "se",
                            'lemma': "se",
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''})


                    else:
                        tok_objects.append({
                            'tok': lemma.strip(),
                            'lemma': lemma.strip(),
                            'gloss': lemma_gloss[1].strip() if len(lemma_gloss) > 1 else ''}
                        )


                        if '-ig' in tok_obj['tok'] and '-ig' in tok_obj['lemma']:
                            tok_obj['gloss'] = 'petit'

                        if 'numéraux' in tok_obj['lemma']:
                            tok_obj['lemma'] = tok_obj['tok']

                        if 'cardinal' in tok_obj['lemma'] and 'cardinal' not in tok_obj['tok']:
                            tok_obj['lemma'] = tok_obj['tok']
                            pattern = r"[^\w\s]+"
                            tok_obj['lemma'] = re.sub(pattern, "", tok_obj['lemma'])






                    tok_n += 1





    return tok_objects

In [10]:
relinks = re.compile(r"\[\[(.*?)\]\]")

def get_wikicode(title):
    wikicode=''
    # on traite les redirections vers d'autres pages
    if '#REDIRECTION' in pages.get(title, ''):             
        #on suit la redirection
        newtitle = relinks.search(pages[title]).group(1)                
        newtitle = newtitle[0].upper()+newtitle[1:]
        if '_' in newtitle: 
            newtitle = newtitle.replace("_", " ")            
        #et on regarde à nouveau si le titre est dans le dictionnaire pages
        if newtitle in pages:
            wikicode = pages[newtitle]              
        else: 
            newtitle = newtitle[0].upper()+newtitle[1:].replace(' ','_')
            if newtitle in pages:
                wikicode = pages[newtitle]
            else:
                newtitle = newtitle.split(',')[0]
                newtitle = newtitle[0].upper()+newtitle[1:]
                if newtitle in pages:
                    wikicode = pages[newtitle]
                else:
                    wikicode='__currentPage:\nstrange redirect to page that does not exist: '+newtitle
                    print(wikicode+'\n')

    else:
        wikicode = pages.get(title, '')  
    return wikicode

In [11]:
import re
import pickle

with open('Pages.pickle', 'rb') as f:
    pages = pickle.load(f)
open('pageTitles.txt','w', encoding="utf-8").write('\n'.join(pages.keys()))
print('extracted',len(pages),'pages')

extracted 9163 pages


In [12]:
rePOS = re.compile(r'articles?|art|verbes?|auxiliaires?|copules?|adverbes?|complémenteurs?|conjonctions?|prépositions?|adjectifs?|noms?|particules verbales?|interjections?|postpositions?|déterminants?|quantifieurs?|pronoms?|noms? propres?|suffixe|interrogatifs?|préfixes?|modaux|pluriels?|indéfinis?|particules? de discours|finales?|exclamatifs?', re.IGNORECASE)

lefffl = [li.strip() for li in open('lefff-2.1.txt').read().split('\n') if li.strip() and li[0]!='#']
lefff = {li.split('\t')[0]:li.split('\t')[2] for li in lefffl}
# print(lefff)


def add_pos(tok_obj):
    if 'pos' in tok_obj and tok_obj['pos'] : return tok_obj # pos already there

    # is the pos in the lemma?
    if 'lemma' in tok_obj:
        m = rePOS.search(tok_obj['lemma'])
        if m:
            tok_obj['pos'] = m.group(0)
            return tok_obj

    
    # is the pos in the gloss?
    if 'gloss' in tok_obj:
        m = rePOS.search(tok_obj['gloss'])
        if m:
            tok_obj['pos'] = m.group(0)
            return tok_obj

    
    # let's check the page for the lemma
    wikicode = get_wikicode(str.capitalize(tok_obj.get('lemma',''))) # TODO: improve as this might fail sometimes
    # print(wikicode[:222])
    m = rePOS.search(wikicode)
    if m:
        tok_obj['pos'] = m.group(0)
        return tok_obj
    
    # let's use the gloss to make an educated guess about the pos using lefff
    if tok_obj.get('gloss','') in lefff:
        tok_obj['pos'] = lefff[tok_obj.get('gloss','')]
    
    return tok_obj

    


In [13]:
import ast
import re


def generate_conll_output(tok_objects, text, text_fr, sent_id):
    conll_output = ""
    conll_id = 1
    inside_id = 1

    for tok_obj in tok_objects:
        if isinstance(tok_obj, dict) and len(tok_obj) == 1:
            field_key, field_value = next(iter(tok_obj.items()))
            conll_line = f"{conll_id}\t{field_key}\t{field_value}\t_\t_\t_\t_\t_\t_\t_\n"
        else:
            if 'multiple' not in tok_obj:
                if 'pos' in tok_obj and 'phonetic' in tok_obj :
                    if tok_obj['phonetic'] != 'None':
                        conll_line = f"{conll_id}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t{tok_obj['pos']}\t_\t{tok_obj['phonetic']}\t_\t_\t_\tgloss={tok_obj['gloss']}\n"
                    else:
                        conll_line = f"{conll_id}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t{tok_obj['pos']}\t_\t_\t_\t_\t_\tgloss={tok_obj['gloss']}\n"

                    conll_id += 1
                    inside_id = conll_id
                else:
                    conll_line = f"{conll_id}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t_\t_\t_\t_\t_\t_\tgloss={tok_obj['gloss']}\n"
                    conll_id += 1
                    inside_id = conll_id
            else:
                conll_line = f"{inside_id}-{inside_id + tok_obj['multiple'] - 1}\t{tok_obj['tok']}\t{tok_obj['lemma']}\t_\t{tok_obj['phonetic']}\t_\t_\t_\t_\t_\n"

            conll_line = re.sub(']]', ' ', conll_line)
            conll_line = re.sub('\[\[kaout', 'kaout', conll_line)
            conll_line = re.sub("'''", '', conll_line)
            conll_line = re.sub('\[','',conll_line)

        conll_output += conll_line.strip() + "\n"

    conll_output = f"# sent_id = None__{sent_id}\n" + \
                   f"# text = {text.replace('||', '')}\n" + \
                   f"# text_fr = {text_fr}\n" + \
                   f"# status = WIP\n" + \
                   conll_output + "\n"

    return conll_output

In [15]:
import re

def process_tokens_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    conll_output = ""
    sent_id = 1  # Initialize sent_id to 1

    with open("couple.txt", "r") as couple_file:
        couple_lines = couple_file.readlines()
        couple_lines = [line.strip() for line in couple_lines]

    for i, line in enumerate(lines):
        tok_objects = ast.literal_eval(line)
        text = couple_lines[i * 4 + 1].replace("text", "").strip()
        text_fr = couple_lines[i * 4 + 3].replace("translation", "").strip()
        text = re.sub("'''",'',text)

        # Update the sent_id value for each example


        conll_output += generate_conll_output(tok_objects, text, text_fr,sent_id)
        sent_id += 1  # Increment sent_id for the next example

    return conll_output


output = process_tokens_file('nt.txt')
print(output)

KeyError: 'pos'